In [1]:
from setting.db import SessionLocal

from graph.graph_knowledge_base import GraphKnowledgeBase, SearchAction

gkb = GraphKnowledgeBase("entities_150001", "relationships_150001", "chunks_150001")
session = SessionLocal()

In [2]:
query = "Where can I find the latest version of tidb"
model_kwargs = {}

In [3]:
from llm_inference.base import LLMInterface
from graph.query_analyzer import DeepUnderstandingAnalyzer

llm_client = LLMInterface("openai", "gpt-4o")
analyzer = DeepUnderstandingAnalyzer(llm_client)
analysis_res = analyzer.perform(query)
print(analysis_res)

Analysis Result:
Reasoning: To deeply analyze the query 'How to find the latest version of tidb' using first principles thinking, we start by understanding the user's fundamental need. The user is likely seeking the most recent version of TiDB, a distributed SQL database, to ensure they have access to the latest features, improvements, and security updates. This need arises from the broader context of software maintenance and development, where staying updated is crucial for performance, security, and compatibility. The fundamental components needed to answer this question include identifying reliable sources where TiDB versions are published, understanding how versioning works, and knowing how to verify the authenticity and recency of the information. The user is asking this question because they want to ensure they are using the most current and supported version of TiDB, which is essential for optimal operation and support. We would know if we've fully answered the question if the u

In [4]:
action_history = []
current_findings = []
docs = {}

next_actions = [SearchAction(
    tool="retrieve_documents",
    query=a
) for a in analysis_res.initial_queries]

reasoning = analysis_res.reasoning
queries = analysis_res.initial_queries

In [5]:

knowledge_retrieved = {}
for action in next_actions:
    if action.tool == 'retrieve_knowledge':
        data = gkb.retrieve_graph_data(session, action.query)
    elif action.tool == 'retrieve_neighbors':
        data = gkb.retrieve_neighbors(session, action.entity_ids, action.query)
    elif action.tool == 'retrieve_documents':
        data = gkb.retrieve_documents(session, action.query, 10)
    else:
        raise ValueError(f"Invalid tool: {action.tool}")

    for doc_id, doc in data.documents.items():
        if doc_id not in knowledge_retrieved:
            knowledge_retrieved[doc_id] = doc
        
        for chunk_id, chunk in doc.chunks.items():
            if chunk_id not in knowledge_retrieved[doc_id].chunks:
                knowledge_retrieved[doc_id].chunks[chunk_id] = chunk
                continue

            existing_chunk = knowledge_retrieved[doc_id].chunks[chunk_id]
            rel_dict = {r['id']: r for r in existing_chunk.relationships}
            for relationship in chunk.relationships:
                rel_id = relationship.id
                if rel_id in rel_dict:
                    rel_dict[rel_id]['similarity_score'] = max(
                        rel_dict[rel_id]['similarity_score'],
                        relationship.similarity_score
                    )
                else:
                    rel_dict[rel_id] = relationship.to_dict()

            knowledge_retrieved[doc_id].chunks[chunk_id].relationships = list(rel_dict.values())

action_history.append(action)

knowledge_retrieved

{15758: DocumentData(id=15758, chunks={}, content='---\ntitle: TiDB 1.0.2 Release Notes\nsummary: TiDB 1.0.2 was released on November 13, 2017. Updates include optimized cost estimation for index point query, support for Alter Table Add Column syntax, and improved query optimization. Placement Driver (PD) scheduling stability was enhanced, and TiKV now supports table splitting and limits key length to 4 KB. Other improvements include more accurate read traffic statistics and bug fixes for LIKE behavior and do_div_mod bug.\n---\n\n# TiDB 1.0.2 Release Notes\n\nOn November 13, 2017, TiDB 1.0.2 is released with the following updates:\n\n## TiDB\n\n- Optimize the cost estimation of index point query\n- Support the `Alter Table Add Column (ColumnDef ColumnPosition)` syntax\n- Optimize the queries whose `where` conditions are contradictory\n- Optimize the `Add Index` operation to rectify the progress and reduce repetitive operations\n- Optimize the `Index Look Join` operator to accelerate th

In [6]:
from graph.knowledge_synthesizer import KnowledgeSynthesizer

synthesizer = KnowledgeSynthesizer(llm_client)
result = synthesizer.iterative_answer_synthesis(
    query=query,
    documents=knowledge_retrieved,
    reasoning=reasoning
)

# Access the results
final_answer = result["final_answer"]
evolution = result["evolution_history"]

Processing document(15758, https://docs.pingcap.com/tidb/v8.1/release-1.0.2)
Update data: {'updated_answer': "To find the latest version of TiDB, you should visit the official TiDB GitHub repository or the TiDB website. These platforms typically provide the most up-to-date information on releases. On GitHub, you can navigate to the 'Releases' section to see the latest version available. Additionally, the TiDB website often has a 'Download' or 'Releases' page that lists the current stable version. As of the information available, TiDB 1.0.2 was released on November 13, 2017, but this is likely outdated. Always check the official sources for the most recent version.", 'changes_made': 'Since the document provided information about TiDB 1.0.2, which is from 2017, I included instructions on how to find the latest version by checking the official TiDB GitHub repository and website. This ensures the user can access the most current version, as the document itself is outdated.', 'temporal_reas

KeyboardInterrupt: 

In [ ]:
final_answer